In [15]:
import matplotlib.pyplot as plt
from numpy import zeros, arange, fft, linspace, pi, cos, sin, sqrt, array, log10, amax, linspace, log10
from pathlib import Path
from os import listdir
from os.path import splitext
import ipywidgets as widgets
from IPython.display import display
from scipy.signal import TransferFunction, bode, butter, filtfilt, freqs
%matplotlib widget

In [2]:
texts0 = [[], []] # X(sequence) and Y(ecg signal) value

########################
files = listdir(Path().absolute())
temp = []
for file in files:
    if splitext(file)[1] == ".txt":
        temp.append(file)
files = temp

#########################
select_signal = widgets.Select(
    options=files,
    description='Select First File:',
    disabled=False
)
signal_length = widgets.Text(value='0', disabled=True)

########################
button_select_signal = widgets.Button(description='Select Signal')
def on_button_select_signal_clicked(change):
    with open(select_signal.get_interact_value(), "r") as file:
        text_file = file.readlines()
    texts0[0] = arange(len(text_file))
    try:
        texts0[1] = [float(x.replace("\n", "")) for x in text_file]
    except ValueError:
        print("Please select single column data file. File Read Error!")
        return
    
    with out_2:
        global plot1
        plot1.set_data(texts0[0], texts0[1])
        fig0_ax.set_xlim([0, len(texts0[0])])
        fig0_ax.set_ylim([min(texts0[1]), max(texts0[1])])
        
    signal_length.value = f"{len(texts0[0])}"
    with out_3:
        out_3.clear_output()
        print("Signal Selected!")  
button_select_signal.on_click(on_button_select_signal_clicked)

########################
out_1 = widgets.Output()
with out_1:
    print("Please select a signal and frequency sampling")


########################
out_2 = widgets.Output()
with out_2:
    fig0, fig0_ax = plt.subplots(nrows=1, ncols=1)
    fig0_ax.set(xlabel="sequence", ylabel="amplitude", title="Signal")
    plot1, = fig0_ax.plot([0,1,2], [0,0,0])
    plt.tight_layout()
    
########################    
out_3 = widgets.Output()


vertical_box_1 = widgets.VBox([out_1, select_signal, signal_length, button_select_signal, out_3])
horizontal_box_1 = widgets.HBox([vertical_box_1, out_2])
display(horizontal_box_1)


In [4]:
def frequency_response_pole_zero(radius, teta, frequency_sampling):
    frequency_response = []
    frequency_start = 0
    frequency_stop = frequency_sampling
    x_position = radius * cos(teta * pi / 180)
    y_position_1 = radius * sin(teta * pi / 180)
    y_position_2 = -radius * sin(teta * pi / 180)
    
    for i in range(frequency_sampling):
        w = 2 * pi * i / frequency_sampling
        numerator = sqrt((1 - cos(2 * w)) ** 2 + (sin(2 * w)) ** 2)
        denominator = sqrt((1 - 2 * x_position * cos(w)
                            + (radius ** 2) * cos(2 * w)) ** 2
                           + (2 * x_position * sin(w)
                              - (radius ** 2) * sin(2 * w)) ** 2)
        frequency_response += [numerator / denominator]
        
    return x_position, y_position_1, y_position_2, frequency_response

def pole_zero_filter(data, radius, teta):
    x_position = radius * cos(teta * pi / 180)
    result = []
    for i in range(len(data)):
        if i <= 2:
            result += [0]
            continue
        result += [2 * x_position * result[i - 1] - (radius ** 2) * result[i - 2] + data[i]]
        if i > 2:
            result[i] -= data[i - 2]
    return result / amax(result)
    

########################
int_frequency_sampling = widgets.IntText(
    value=100,
    description='Fs:',
    disabled=False
)
        
########################
float_radius = widgets.FloatText(
    value=0.5,
    step=0.1,
    description='Radius:',
    disabled=False
)

########################
int_teta = widgets.IntText(
    value=30,
    description='Teta:',
    disabled=False
)

########################
button_pole_zero = widgets.Button(description='Plot Pole Zero')
def on_button_pole_zero_clicked(change):
    x_position, y_position_1, y_position_2, frequency_response = frequency_response_pole_zero(
            float_radius.get_interact_value(), 
            int_teta.get_interact_value(), 
            int_frequency_sampling.get_interact_value())
    result_pole_zero = pole_zero_filter(texts0[1], 
                                        float_radius.get_interact_value(), 
                                        int_teta.get_interact_value())
    
    with out_4:
        plot2.set_data(arange(len(frequency_response)//2),
                       frequency_response[:len(frequency_response)//2])
        fig1_ax.set_xlim([0, len(frequency_response)//2])

        
    with out_5:
        plot4.set_data([x_position], [y_position_1])
        plot5.set_data([x_position], [y_position_2])
        
    with out_6:
        plot6.set_data(texts0[0], result_pole_zero)
        fig3_ax.set_xlim([0, len(texts0[0])])
        fig3_ax.set_ylim([min(result_pole_zero), max(result_pole_zero)])
button_pole_zero.on_click(on_button_pole_zero_clicked)

########################
out_4 = widgets.Output()
with out_4:
    fig1, fig1_ax = plt.subplots(nrows=1, ncols=1)
    fig1_ax.set(xlabel="hz", ylabel="gain", title="Frequency Response", yscale= "log")
    fig1_ax.set_ylim([10**(-1), 10**(1)])
    plot2, = fig1_ax.plot([0,1,2], [1,1,1])
    plt.tight_layout()
    
########################    
out_5 = widgets.Output()
with out_5:
    fig2, fig2_ax = plt.subplots(nrows=1, ncols=1)
    fig2_ax.set(xlabel="xaxis", ylabel="yaxis", title="Pole Zero Position")
    plot3, = fig2_ax.plot(cos(linspace(0, 2 * pi, 100)), sin(linspace(0, 2 * pi, 100)), color="red")
    plot4, = fig2_ax.plot([0], [0], marker="x", color="blue")
    plot5, = fig2_ax.plot([0], [0], marker="x", color="blue")
    fig2_ax.set_xlim([-1.2, 1.2])
    fig2_ax.set_ylim([-1.2, 1.2])
    fig2_ax.set_aspect('equal')
    plt.tight_layout()
    
out_6 = widgets.Output()
with out_6:
    fig3, fig3_ax = plt.subplots(nrows=1, ncols=1)
    fig3_ax.set(xlabel="sequence", ylabel="amplitude", title="Filter Result")
    plot6, = fig3_ax.plot([0,1,2], [1,1,1])
    plt.tight_layout()
    
horizontal_box_2 = widgets.HBox([int_frequency_sampling, float_radius, int_teta, button_pole_zero])
horizontal_box_3 = widgets.HBox([out_4, out_5])
display(horizontal_box_2, horizontal_box_3, out_6)

Output()

In [19]:
# def low_pass_filter(data, frequency_sampling, type, frequency_cutoff, orde):
#     nyquist_frequency_sampling = 0.5 * frequency_sampling
#     constant = frequency_cutoff / nyquist_frequency_sampling
#     b, a = butter(2, constant, type, analog=False)
#     w, h = freqs(b, a)
#     result = filtfilt(b, a, data)
#     return w, 20 * log10(abs(h)), result

def frequency_response_low_pass(frequency_sampling, frequency_cutoff):
    nyquist_frequency_sampling = 0.5 * frequency_sampling
    constant = frequency_cutoff / nyquist_frequency_sampling
    numerator, denominator = butter(2, constant, "lowpass", analog=False)
    normalized, frequency_response = freqs(numerator, denominator)
    return normalized, frequency_response

def low_pass_filter(data):
    result = []
    for i in range(len(data)):
        if i <= 2:
            result += [data[i]]
            continue
        if i <= 6:
            result += [2*result[i-1] - result[i-2] + data[i]]
            continue
        if i <= 12:
            result += [2*result[i-1] - result[i-2] + data[i] - 2*data[i-6]]
            continue
        result += [2*result[i-1] - result[i-2] + data[i] - 2*data[i-6] + data[i-12]]
    return result

########################
int_frequency_sampling = widgets.IntText(
    value=100,
    description='Fs:',
    disabled=False
)

########################
int_frequency_cutoff= widgets.IntText(
    value=10,
    description='Fc:',
    disabled=False
)

########################
button_low_pass = widgets.Button(description='Plot Low Pass')
def on_button_low_pass_clicked(change):
    result_low_pass = low_pass_filter(texts0[1])
    normalized, frequency_response = frequency_response_low_pass(int_frequency_sampling.get_interact_value(), 
                                                     int_frequency_cutoff.get_interact_value())
    
    with out_7:
        plot7.set_data(normalized,
                       20 * log10(abs(frequency_response)))
        fig4_ax.set_xlim([min(normalized), max(normalized)])   
        fig4_ax.set_ylim([min(20 * log10(abs(frequency_response))), max(20 * log10(abs(frequency_response)))])  
        
    with out_8:
        plot8.set_data(texts0[0], result_low_pass)
        fig5_ax.set_xlim([0, len(texts0[0])])
        fig5_ax.set_ylim([min(result_low_pass), max(result_low_pass)])
button_low_pass.on_click(on_button_low_pass_clicked)

########################
out_7 = widgets.Output()
with out_7:
    fig4, fig4_ax = plt.subplots(nrows=1, ncols=1)
    fig4_ax.set(xlabel="hz", ylabel="gain", title="Frequency Response", xscale= "log")
    fig4_ax.set_ylim([10**(-1), 10**(1)])
    plot7, = fig4_ax.plot([0,1,2], [1,1,1])
    plt.tight_layout()
    
########################
out_8 = widgets.Output()
with out_8:
    fig5, fig5_ax = plt.subplots(nrows=1, ncols=1)
    fig5_ax.set(xlabel="sequence", ylabel="amplitude", title="Filter Result")
    plot8, = fig5_ax.plot([0,1,2], [1,1,1])
    plt.tight_layout()

horizontal_box_4 = widgets.HBox([int_frequency_sampling, int_frequency_cutoff, button_low_pass])
horizontal_box_5 = widgets.HBox([out_7, out_8])
display(horizontal_box_4, horizontal_box_5)